<h1 style = "text-align: center; ">Descriptive Title</h1>
<h2 style = "text-align: center; ">ST445 - Managing and Visualizing Data</h2>
<h3 style = "text-align: center; ">Candidate IDs: 38682, XXXXX, YYYYY</h3>


### I. Notebook preparation (maybe this section is not needed)

Perhaps we include something similar to this example from "Example 2"

[[Before running this notebook, please make sure you have all necessary modules installed in your environment. Potentially less common modules used include:

google.cloud
dotenv
networkx
geopandas
praw
transformers
plotly.graph_objects
ipywidgets
folium
As usual, they can be installed by running the command pip install [module] in the terminal.

Furthermore, please make sure your Python version is compatible with all the modules. While writing this, it became apparent there might be some compatibility issues with newer Python versions (especially 3.11 and newer). In case you run into any issues, it might be worth trying to run the code with an older version such as Python 3.9.]]

Our complete GitHub repository can be found at the following location: https://github.com/lse-st445/2024-project-data-knows-ball [[Should we put this in the title of our paper??]]

In [1]:
# Import relevant packages
import requests
from bs4 import BeautifulSoup

# Install lxml with conda install anaconda::lxml to use HMTL and XML with Python

### II. Introduction and data description

[[Describe our data sets and pose our research question]]

[[Maybe include data dictionaries of some sort similar to Table 1.3.1 and Table 1.3.2 in "Example 2"]]

### III. Data acquisition

In [5]:
# API 



In [16]:
# Webscrapping UK unemployment data
url_uk_unemp = "https://www.ons.gov.uk/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsx/lms"

page = requests.get(url_uk_unemp)

soup = BeautifulSoup(page.content, "lxml")
# print(soup.prettify())

table_rows = soup.find_all("tr")


for row in table_rows:
    print(row.text.strip())

Period
Value
Period
Value
1971
4.1
1972
4.3
1973
3.7
1974
3.7
1975
4.5
1976
5.4
1977
5.6
1978
5.5
1979
5.4
1980
6.8
1981
9.6
1982
10.7
1983
11.5
1984
11.8
1985
11.4
1986
11.3
1987
10.4
1988
8.6
1989
7.2
1990
7.1
1991
8.9
1992
9.9
1993
10.4
1994
9.5
1995
8.6
1996
8.1
1997
6.9
1998
6.2
1999
6.0
2000
5.4
2001
5.1
2002
5.2
2003
5.0
2004
4.8
2005
4.8
2006
5.4
2007
5.3
2008
5.7
2009
7.6
2010
7.9
2011
8.1
2012
7.9
2013
7.6
2014
6.2
2015
5.4
2016
4.9
2017
4.4
2018
4.1
2019
3.8
2020
4.6
2021
4.6
2022
3.8
2023
4.1
1971 Q1
3.8
1971 Q2
4.1
1971 Q3
4.2
1971 Q4
4.4
1972 Q1
4.5
1972 Q2
4.4
1972 Q3
4.3
1972 Q4
4.2
1973 Q1
3.9
1973 Q2
3.7
1973 Q3
3.6
1973 Q4
3.4
1974 Q1
3.6
1974 Q2
3.6
1974 Q3
3.7
1974 Q4
3.7
1975 Q1
4.0
1975 Q2
4.3
1975 Q3
4.7
1975 Q4
5.0
1976 Q1
5.3
1976 Q2
5.4
1976 Q3
5.5
1976 Q4
5.5
1977 Q1
5.5
1977 Q2
5.5
1977 Q3
5.7
1977 Q4
5.7
1978 Q1
5.6
1978 Q2
5.6
1978 Q3
5.5
1978 Q4
5.4
1979 Q1
5.4
1979 Q2
5.3
1979 Q3
5.4
1979 Q4
5.5
1980 Q1
5.8
1980 Q2
6.3
1980 Q3
7.1
1980 Q4
8.0
1981 Q1
8.

In [23]:
table_headers = soup.find_all("th")
table_headers = table_headers[0:2]
table_headers = [t.text.replace("\n", "") for t in table_headers]
table_headers

['Period', 'Value']

In [27]:
row_number=2
row_data=soup.find_all('tr')[row_number].find_all('td')
print(row_data)

p, v = row_data[0:2]
print(p.text, v.text)

[<td>1971</td>, <td>4.1</td>]
1971 4.1


In [ ]:
# Webscrapping UK CPIH data
url_uk_cpih = "https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/l55o/mm23"


### IV. Data preparation

In [7]:
# Clean and merge datasets



### V. Visualizations

[[Description of what visualizations we decided to include and why]]

In [8]:
# Code for visualizations



[[Explanation/interpretation of the visualizations are depicting]]

### VI. Data modeling

### VII. Conclusion